# Custom Nonbonded Potential: Yukawa on rigid bodies

Here we define a custom force class where particles interact through a Yukawa potential and a soft repulsion,

\begin{equation}
w(r) / k_BT = \frac{\lambda_Bz_iz_j}{r}e^{-r/\lambda_D} + 4\beta\epsilon_{ij} \left ( \frac{\sigma_{ij}}{r}\right )^{12}
\end{equation}

where
$\lambda_B=e^2/4\pi\epsilon_0\epsilon_rk_BT$ and
$\lambda_D=(4\pi\lambda_B\sum \rho_iz_i^2)^{-1/2}$ are the Bjerrum and Debye lengths, respectively. $\rho_i$ is the number density of the $i$th ion.
In this example we also create two rigid bodies using harmonic bonds to constrain the positions.

Some comments:

1. The potential is defined in `CustomNonbonded` is defined in `cg.zml` and must return energy in `kJ/mol`.
2. The Bjerrum and Debye lengths are set via global parameters

In [2]:
%matplotlib inline
from __future__ import print_function
from simtk.openmm import app
import simtk.openmm as mm
from simtk import unit
from sys import stdout, exit
import numpy as np
import math
import mdtraj as mdtraj
from itertools import combinations

cutoff           = 50*unit.angstrom
useMinimize      = True
epsilon_r        = 80.
temperature      = 300*unit.kelvin
kT               = unit.BOLTZMANN_CONSTANT_kB*temperature
steps_eq         = 5000
steps_production = 1e5
steps_total      = steps_eq + steps_production

pdb = app.PDBFile('squares.pdb')
forcefield = app.ForceField('yukawa.xml')

system = forcefield.createSystem(pdb.topology, nonbondedMethod=app.CutoffPeriodic, nonbondedCutoff=cutoff )
    
# set Bjerrum length and inverse Debye length
for force in system.getForces():
  if isinstance(force, mm.CustomNonbondedForce):
    nonbonded = force
    for i in range(nonbonded.getNumGlobalParameters()):
        name=nonbonded.getGlobalParameterName(i)
        if name=='lB':    nonbonded.setGlobalParameterDefaultValue(i, 0.7*unit.nanometer );
        if name=='kappa': nonbonded.setGlobalParameterDefaultValue(i, 0);

def atomIndexInResidue(residue):
    """ list of atom index in residue """
    index=[]
    for a in list(residue.atoms()):
      index.append(a.index)
    return index

def uniquePairs(index):
    """ list of unique, internal pairs """
    return list(combinations( range(index[0],index[-1]+1),2 ) )

def addHarmonicConstraint(harmonicforce, pairlist, positions, threshold, k):
    """ add harmonic bonds between pairs if distance is smaller than threshold """
    for i,j in pairlist:
      distance = unit.norm( positions[i]-positions[j] )
      if distance<threshold:
        harmonicforce.addBond(i,j,distance,k)
        print("added harmonic bond between", i, j, 'with distance',distance)

# add bonds between atoms in residues
harmonic = mm.HarmonicBondForce()
system.addForce(harmonic)
threshold = 6.0*unit.angstrom
k         = 10*kT*unit.AVOGADRO_CONSTANT_NA/unit.angstrom**2
for residue in pdb.topology.residues():
    index    = atomIndexInResidue(residue)
    pairlist = uniquePairs(index)
    addHarmonicBonds(harmonic, pairlist, pdb.positions, threshold, k)
    for i,j in pairlist:
        nonbonded.addExclusion(i,j)
                        
integrator = mm.LangevinIntegrator(
    temperature,
    1.0/unit.picoseconds, 
    20*unit.femtoseconds)

#integrator.setConstraintTolerance(0.0001)

simulation = app.Simulation(pdb.topology, system, integrator)
simulation.context.setPositions(pdb.positions)

if useMinimize:
  print('Minimizing...')
  simulation.minimizeEnergy()

print('Equilibrating...')
simulation.context.setVelocitiesToTemperature(300*unit.kelvin)
simulation.step(steps_eq)

simulation.reporters.append(mdtraj.reporters.HDF5Reporter('trajectory.h5', 100))
simulation.reporters.append(app.StateDataReporter(stdout, int(steps_total/10), step=True, 
    potentialEnergy=True, temperature=True, progress=True, remainingTime=False, 
    speed=True, totalSteps=steps_total, volume=True, separator='\t'))

print('Production...')
simulation.step(steps_production)

print('Done!')

added harmonic bond between 0 1 with distance 0.4 nm
added harmonic bond between 0 2 with distance 0.565685424949 nm
added harmonic bond between 0 3 with distance 0.4 nm
added harmonic bond between 1 2 with distance 0.4 nm
added harmonic bond between 1 3 with distance 0.565685424949 nm
added harmonic bond between 2 3 with distance 0.4 nm
added harmonic bond between 4 5 with distance 0.4 nm
added harmonic bond between 4 6 with distance 0.565685424949 nm
added harmonic bond between 4 7 with distance 0.4 nm
added harmonic bond between 5 6 with distance 0.4 nm
added harmonic bond between 5 7 with distance 0.565685424949 nm
added harmonic bond between 6 7 with distance 0.4 nm
Minimizing...
Equilibrating...
Production...
#"Progress (%)"	"Step"	"Potential Energy (kJ/mole)"	"Temperature (K)"	"Box Volume (nm^3)"	"Speed (ns/day)"
10.0%	10500	-5.09292711131	216.455329723	1000.0	0
20.0%	21000	3.95787717402	404.266466826	1000.0	6.18e+03
30.0%	31500	-4.1702266885	327.527084784	1000.0	6.07e+03
40.0%	